# Cambio de los valores de NDVI a lo largo del tiempo
**Autora: Pamela E. Pairo**

El siguiente documento muestran cómo calcular y visualizar los cambios del NDVI a lo largo del tiempo de campos agrícolas de interés.

## En un único campo agrícola

Se realiza primero la carga de las librerias necesarias y se definen una serie de funciones que se usarán a lo largo del documento.

In [ ]:
import os
import ee
import geemap
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

def getNDVI(image):
    return image.addBands(image.normalizedDifference(['B8','B4']).rename('NDVI'))

def make_point_mean (fechaInicio, fechaFin, poi):
    
    collection = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(poi) \
    .filterDate(fechaInicio,fechaFin) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .select('B3','B4', 'B5', 'B6','B7', 'B8', 'B8A') \
    .map(getNDVI)
    
    def poi_mean(img):
        mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=10).get('NDVI')
        return img.set('date', img.date().format()).set('mean', mean)
    
    nested_list= collection.map(poi_mean).reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)
    df= pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])
    df['date'] = pd.to_datetime(df['date'])
    return df

def NDVI_categorized(x):
    if -1 < x <= 0.1:
        return 'Nube'
    elif 0.1 < x <= 0.3:
        return 'Sin Actividad'
    elif 0.3 < x <= 0.55:
        return 'Actividad Moderada'
    return 'Actividad Alta'

In [ ]:
# Authenticate the Google earth engine with google account
ee.Initialize()

Se selecciona un campo agrícola en el mapa, se dibuja un punto en el mismo y luego se extrae la información de su latitud y longitud. Luego, se genera el `poi`, el cual es un circulo de radio a definir (en este ejemplo se selecciona un radio de 50m) cuyo centro está dado por las coordenadas del punto en el campo agrícola. Para tener resultados más claros y con menos ruido, es importante localizar `poi` en un área homogenea y no por ejemplo posicionar `poi` en el borde de dos campos agrícolas consecutivos.

In [ ]:
#campo de trigo en la campaña de invierno de 2019, según información provista por INTA
poi = ee.Geometry.Point([-60.4369,-33.9438]).buffer(50)

A continuación se muestra la localización de `poi` y la imagen Sentinel-2 de Septiembre de 2019 en una combinación de bandas 11-8-3.

In [ ]:
paises = ee.FeatureCollection("FAO/GAUL/2015/level2")
pais = paises.filter(ee.Filter.eq('ADM0_NAME', 'Argentina'))
prov = pais.filter(ee.Filter.eq('ADM1_NAME', 'Buenos Aires'))
Pergamino = prov.filter(ee.Filter.eq('ADM2_NAME', 'Pergamino'))

fechaInicio = '2019-09-01'
fechaFin = '2019-09-16'

image = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(Pergamino) \
    .filterDate(fechaInicio,fechaFin) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .select('B2', 'B3','B4', 'B5', 'B6','B7', 'B8', 'B8A', 'B11') \
    .map(lambda image: image.clip(Pergamino))

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B11','B8', 'B3']
}
Map = geemap.Map()
Map.centerObject(Pergamino, 10)
Map.addLayer(image, vis_params, "Sep 2019-Sentinel2")
Map.addLayer(poi, {}, 'poi')
Map

Luego, se selecciona imágenes Sentinel-2 de un rango de fechas y se calcula la media del NDVI dentro `poi` para cada imagen. Posterirmente, se realiza la categorización de los valores de NDVI.

In [ ]:
fechaInicio = '2018-06-16'
fechaFin = '2021-08-20'
df_p1= make_point_mean (fechaInicio, fechaFin, poi)
df_p1['categ'] = df_p1['mean'].apply(NDVI_categorized)

Se realiza un gráfico para visualizar los resultados obtenidos.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

sns.scatterplot(data=df_p1, 
                x="date", 
                y="mean", 
                hue= "categ", 
                hue_order=['Actividad Alta', 'Actividad Moderada', 'Sin Actividad', 'Nube'])

ax.set_ylabel('NDVI',fontsize=20)
ax.set_xlabel('Fecha',fontsize=20)
ax.set_title('Cambio de los valores de NDVI para un campo agrícola \n (Junio 2016- Agosto 2021)', fontsize=20)
ax.set_ylim([0, 0.9])
plt.legend(title= "Actividad fotosintética", loc="upper left")

plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2)) 
plt.gca().xaxis.set_minor_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.gcf().autofmt_xdate()

## En varios campos agrícolas a la vez

En ocasiones resulta conveniente visualizar los cambios del NDVI en varios campos agrícolas a la vez. Primero, se definen los puntos a evaluar. En este caso, se establecieron dos campos agrícolas y un sitio ribereño como punto de control.

In [ ]:
control_ribera = ee.Geometry.Point([-60.2389,-33.7471]).buffer(50)# sitio ribereño
poi_1 = ee.Geometry.Point([-60.2237,-33.7949]).buffer(50)#campo de trigo en septiembre de 2019 según Inta
poi_2 = ee.Geometry.Point([-60.2882,-33.6918]).buffer(50)#campo de trigo en septiembre de 2019 según Inta

#Se ponen todos los puntos en una lista
puntos = [control_ribera, poi_1, poi_2]

Para cada punto de la lista, se calcula el NDVI en el período especificado y se arma un único _dataframe_

In [ ]:
fechaInicio = '2018-06-16'
fechaFin = '2021-08-20'

list_of_dfs = []
num = -4

for point in puntos:
    calc=make_point_mean (fechaInicio, fechaFin, point)
    list_of_dfs.append(calc)
    for punto in list_of_dfs:
        num += 1
        punto['Punto']= f'Campo {num}'
    df_puntos = pd.concat(list_of_dfs).reset_index(drop=True)

Para una mejor visualización de los resultados, la temporada de verano es representada como franjas de color verde claro. "Campo 0" corresponde al sitio de control (sitio ribereño).

In [ ]:
a = '2018-10'
b = '2019-04'
c= '2019-10'
d= '2020-04'
e= '2020-10'
f= '2021-04'

sns.relplot(data=df_puntos, x = 'date', y = 'mean', kind = 'line', hue = 'Punto',\
            palette = ['red', 'blue', "green"],\
            height=5, aspect=2.3)
plt.axvspan(a, b, color='y', alpha=0.5, lw=0)
plt.axvspan(c, d, color='y', alpha=0.5, lw=0)
plt.axvspan(e, f, color='y', alpha=0.5, lw=0)

plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2)) 
plt.gca().xaxis.set_minor_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.gcf().autofmt_xdate()
plt.xlabel(" ")
plt.ylabel("NDVI medio", fontsize=10)
#plt.savefig('NDVI_evolución.jpg', dpi=160)#descomentar para guardar el grafico como .jpg